# Importing Libraries

In [7]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

# Data Fetching

In [8]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16],low_memory=False)
    out= pd.read_csv('../../../data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19],low_memory=False)
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


# A fast algorithm for Independent Component Analysis Decomposition

In [9]:
from sklearn.decomposition import SparsePCA
import warnings
scaler_obj1=SparsePCA()
scaler_obj2=SparsePCA()
X1=scaler_obj1.fit_transform(A1)
Y1=scaler_obj2.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

# Parameter Tuning

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import RandomizedSearchCV

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [7,8,10,],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'n_estimators' : [80,90,100,200,400,1000],
        'objective': ['reg:squarederror','count:poisson']
    }

    xgb_model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)

    gsearch = RandomizedSearchCV(estimator = xgb_model, param_distributions = param_tuning, n_iter = 5, cv = 2, verbose=2, random_state=0, n_jobs = -1)

    
    grid_result = MultiOutputRegressor(gsearch).fit(x_train, y_train)

    return grid_result.estimators_[0].best_params_

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

params=hyperParameterTuning(x_train,y_train)
params

# Model

In [10]:
# Splitting Data into training and testing dataset
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)

from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

#creating object of sgboostregressor
model1=MultiOutputRegressor(XGBRegressor(tree_method='gpu_hist', gpu_id=0,max_depth=8,cosample_bytree=.5,learning_rate=.1,min_child_weight=3,
                   n_estimators=200,subsample=.7))

#training the model
model_fit1=model1.fit(x_train, y_train)
print("Model training is Done!!")

#saving as file
filename1 = 'xgboost.sav'
pickle.dump(model_fit1, open(filename1, 'wb'))

[18:59:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:00:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:01:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "cosample_bytree" } might not be use

# Error Analysis

In [11]:
from sklearn.metrics import r2_score
from sklearn import metrics
train_sizes=['NO2','O3','NO','CO','PM1','PM2.5','PM10']

#finding out the r2 score
y_train_pred1=model1.predict(x_train)
r2_train1=r2_score(y_train,y_train_pred1,multioutput='variance_weighted')

y_test_pred1=model1.predict(x_test)
r2_test1=r2_score(y_test,y_test_pred1,multioutput='variance_weighted')

print('r2 score on train data '+str(r2_train1))
print('r2 score on test data '+ str(r2_test1))

xgboost_mae=metrics.mean_absolute_error(y_test, y_test_pred1)
xgboost_mse=metrics.mean_squared_error(y_test, y_test_pred1)
xgboost_rmse=np.sqrt(xgboost_mse)
print('Mean Absolute Error:',xgboost_mae)
print('Mean Squared Error:',xgboost_mse )
print('Root Mean Squared Error:',xgboost_rmse)
print(' \n')

r2 score on train data 0.3351657659609297
r2 score on test data -1498.28155851662
Mean Absolute Error: 103.29054
Mean Squared Error: 2103667800.0
Root Mean Squared Error: 45865.758
 



# Prediction of particles

In [12]:
import pickle
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=42)
loaded_model_fit7 = pickle.load(open("xgboost.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
print("Predicted :\n",y_test_pred)
print("\n")
r2_test=r2_score(y_test,y_test_pred,multioutput='variance_weighted')
print("R2 Score : ",r2_test)

Predicted :
 [[-6.3266712e+01  2.6222984e+01  9.5894585e+01 ... -3.2326351e+01
  -3.4267503e-01 -3.8126916e-01]
 [-4.2865105e+01 -4.8326118e+01  2.6734684e+01 ...  3.0773718e+01
  -1.6355041e+00  1.5065483e+00]
 [-5.8958878e+01 -1.1239226e+01 -3.4998175e+02 ... -1.2724620e+01
   3.1426024e-01 -2.6705676e-01]
 ...
 [-8.6125565e+01 -1.4471008e+01 -1.7130704e+02 ...  4.7370903e+01
  -1.6055179e+00  3.3796111e-01]
 [-4.5717514e+01  3.3935410e+01  3.2849377e+02 ... -1.3256995e+01
  -1.0992424e+00  4.0163147e-01]
 [-9.2042099e+01 -2.7857420e+01  4.1238953e+02 ... -1.8715380e+01
   2.5518751e+00 -1.5739179e-01]]


R2 Score :  -1498.28155851662


In [13]:
x_test=scaler_obj1.inverse_transform(x_test)
y_test_pred=scaler_obj2.inverse_transform(y_test_pred)
y_test=scaler_obj2.inverse_transform(y_test)
pd.DataFrame(y_test_pred)

AttributeError: 'SparsePCA' object has no attribute 'inverse_transform'

In [ ]:
import datetime
from datetime import datetime
temp_arr=x_test[0:,1]
dates=list()
for i in range(0,len(temp_arr)):
    datetime_str=str(int(temp_arr[i]))
    datetime_str=datetime_str[0:6]
    if(datetime_str[4:6]=="00"):
        datetime_str=datetime_str[0:4]+'1'
    datetime_obj = datetime.strptime(datetime_str,"%y%m%d")
    dates.append(str(datetime_obj.date()))


In [ ]:
import pandas as pd
df1=pd.DataFrame(y_test,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df1.index.name="DATE"
print("Actual Values:")
df1

In [ ]:
arr=["Y_Actual"]*25
temp_df1=df1.head(25)
temp_df1['Data']=arr
temp_df1.to_excel("xgboost_y_test.xlsx")
temp_df1

In [ ]:
import pandas as pd
df2=pd.DataFrame(y_test_pred,index=dates,columns=['NO2','O3','NO','CO','PM1','PM2.5','PM10'])
df2.index.name="DATE"
print("Predicted Values:")
df2

In [ ]:
arr=["Y_Predicted"]*25
temp_df2=df2.head(25)
temp_df2['Data']=arr
temp_df2.to_excel("xgboost_y_test_pred.xlsx")
temp_df2

In [13]:
#Completed